In [43]:
import sys
import json, os, re, shutil, sys, time
import collections, itertools
import pickle
import numpy as np
from collections import Counter, defaultdict
from datetime import timedelta
from sklearn.metrics import precision_recall_fscore_support

interim_path = '../interim/'

print "Done!"

Done!


In [4]:
samp = True
samp = '_samp' if samp else ''

# pretty print elapsed time
def elapsed_time(start, stop):
    time_str = str(timedelta(seconds = time.time()-start))
    if( time_str.startswith('0:')):
        time_str = time_str[2:]
        if( time_str.startswith('00:')):
            time_str = time_str[3:]
    return 'Time elapsed: %s' % time_str


In [5]:
# load preprocessed data array from file
#   podcast+array[0]: podcast name
#   podcast_array[1]: list of words in concatated descriptions, cleaned 
#   podcast_array[2]: subgenre (label)
with open( interim_path + 'preprocessed_concatenated_all' + samp + '.p') as p:
	podcast_array = pickle.load(p)

print podcast_array.shape

(1241, 3)


In [6]:
# get the number of tokens plus counts for each subgenre 
token_counts_by_class = defaultdict(lambda: Counter())
token_counts = Counter()
start = time.time()
subgenres = []
for podcast in podcast_array:
    document=podcast[1]
    subgenre = podcast[2]
    if subgenre not in subgenres:
        subgenres.append(subgenre)
    for word in document:
        token_counts_by_class[word][subgenre]+=1
        token_counts[word]+=1

print elapsed_time(start, time.time())
print''
print "token_counts_by_class", len(token_counts_by_class)
print "token_counts", len(token_counts)


Time elapsed: 12.773169

token_counts_by_class 174505
token_counts 174505


In [44]:
#log_probabilties of each token being in each subgenre
token_log_probs_by_genre = {}
start = time.time()
for word in token_counts.keys():
    # probability of the word occurring in the class is word_count_in_class / token_count_in_class
    subgenre_probs={}
    for subgenre in subgenres:
        word_cnt = token_counts_by_class[word][subgenre]
        token_cnt = token_counts[word]
        log_prob=0.0
        if token_cnt!=0:
            log_prob = word_cnt/(token_cnt*1.0)

        subgenre_probs[subgenre]=log_prob
        token_log_probs_by_genre[word]=subgenre_probs

print elapsed_time(start, time.time())
print 'token_log_probs_by_genre', len(token_log_probs_by_genre)


Time elapsed: 09.496637
token_log_probs_by_genre 174505


In [45]:
# get the number of tokens plus their counts for each document
token_counts_by_document_list=[]
token_counts_by_document = Counter()
start = time.time()
for podcast in podcast_array:
    document=podcast[1]
    for word in document:
        token_counts_by_document[word]+=1
token_counts_by_document_list.append(token_counts_by_document)
print elapsed_time(start, time.time())


Time elapsed: 05.476432


In [51]:
log_probs = []
start = time.time()
arr = podcast_array

y_true=[]
y_pred=[]

for podcast in arr:
    document=podcast[1]
    subgenre_log_probs = {}
    for word in document:
        # compute the probability for the word being in each subgenre
        for subgenre in subgenres:  
            if subgenre not in subgenre_log_probs:
                subgenre_log_probs[subgenre] = token_log_probs_by_genre[word][subgenre]
            else:
                subgenre_log_probs[subgenre] += token_log_probs_by_genre[word][subgenre]

    max_log_prob=0.0
    dominent_subgenre = ''
    for subgenre in subgenre_log_probs.keys():
        if subgenre_log_probs[subgenre] > max_log_prob:
            max_log_prob = subgenre_log_probs[subgenre]
            dominent_subgenre = subgenre
            
    y_true.append(podcast[2])
    y_pred.append(dominent_subgenre)

print elapsed_time(start, time.time())
precision, recall, f_score, _ = precision_recall_fscore_support(y_true,y_pred,average='macro')
print 'precision: %3.4f, recall: %3.4f, f-score: %3.4f' % ( precision, recall, f_score )

Time elapsed: 02:58.945542
precision: 0.9053, recall: 0.7778, f-score: 0.8053
